In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import os.path as osp
from itertools import cycle

import bokeh.plotting as blt
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(INLINE)
from bokeh.palettes import Inferno11 as palette

In [ ]:
results_dir = "results"
exp_names = []
df_list = []
score_names = set()
for exp_name in os.listdir(results_dir):
    exp_dir = osp.join(results_dir, exp_name)
    if osp.isdir(exp_dir):
        scores_path = osp.join(exp_dir, "scores.csv")
        if osp.exists(scores_path):
            df = pd.read_csv(scores_path)
            if "Unnamed: 0" in df.columns:
                df.drop("Unnamed: 0", axis="columns", inplace=True)
            score_names = score_names.union([col for col in df.columns if not col.startswith("train_")])
            df["exp_name"] = [exp_name] * len(df)
            df_list.append(df)
            exp_names.append("_".join(exp_name.split("_")[1:]))

In [ ]:
bins = 10
for score_name in score_names:
    fig = blt.figure(title=score_name, width=600, height=400)
    color_gen = cycle(palette)
    for exp_name, scores_df in zip(exp_names, df_list):
        test_df = scores_df[[col for col in scores_df.columns if "train_" not in col]]
        train_df = scores_df[[col for col in scores_df.columns if "train_" in col]]
        train_df.columns = [col.replace("train_", "") for col in train_df.columns]
        diff_df = pd.DataFrame()
        if train_df.size != 0 and test_df.size != 0:
            diff_df = train_df - test_df
            
        for exp_name, scores_df in zip(
            [exp_name + "_testset", exp_name + "_trainset", exp_name + "_diff"], [test_df, train_df, diff_df]):
            if scores_df.size != 0 and score_name in scores_df.columns:
                hcounts, hedges = np.histogram(scores_df[score_name], bins=bins)
                hcounts = hcounts / float(hcounts.sum())
                fig.quad(top=hcounts, left=hedges[:-1], right=hedges[1:], bottom=0,
                         color=next(color_gen), alpha=0.6, legend=exp_name)
            
    fig.legend.location = "top_left"
    blt.show(fig)